In [113]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
from kedro.runner.sequential_runner import SequentialRunner
from kedro.io.data_catalog import DataCatalog
from jaffle_shop.utilities.graph import kedro_node_to_graph, kedro_pipeline_to_graph, render_nx_graph

In [116]:
default_pipeline = pipelines['__default__']
nx_pipeline_graph = kedro_pipeline_to_graph(default_pipeline, catalog)
render_nx_graph(
    nx_pipeline_graph, 
    color_category="operation_name",
    label="operation_name",
    description="data_name",
).show('graph.html')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 default_pipeline = pipelines['__default__']                                                  │
│   2 nx_pipeline_graph = kedro_pipeline_to_graph(default_pipeline, catalog)                       │
│ ❱ 3 render_nx_graph(                                                                             │
│   4 │   nx_pipeline_graph,                                                                       │
│   5 │   color_category="operation_name",                                                         │
│   6 │   label="operation_name",                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: render_nx_graph() got an unexpected keyword argument 'color_category'

In [112]:
n = default_pipeline.nodes[2]
nx_node_graph = kedro_node_to_graph(n,catalog)
render_nx_graph(
        nx_node_graph,
        color_field="operation_name",
        label_attr="operation_name",
        title_attr="data_name",
).show('graph.html')

graph.html


<IPython.lib.display.IFrame object at 0x293dfdb70>

In [85]:
import networkx as nx
render_nx_graph(
    nx.disjoint_union(nx_node_graph, nx_pipeline_graph),
    color_field="operation_name",
    label_attr="operation_name",
    title_attr="data_name",
).show('graph.html')


graph.html


<IPython.lib.display.IFrame object at 0x293e04b50>